In [126]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
import easydict

import os
import argparse
import numpy as np

from models import *
from utils import progress_bar
from torch.utils.tensorboard import SummaryWriter



In [127]:
def get_grad_norm_by_param(model):
    return {name: torch.sum(param ** 2) for name, param in model.named_parameters()}

In [128]:
def compute_gradient_norm(model):
    grad_norm = 0
    for p in model.parameters():
        if p.grad is not None:
            grad_norm += (p.grad.data ** 2).sum().item()
    return grad_norm

In [129]:
def compute_gradient_mean(model):
    grad_mean = 0
    for p in model.parameters():
        if p.grad is not None:
            grad_mean += p.grad.data.mean().item()
    return grad_mean

In [130]:
def compute_gradient_mean(model):
    grad_mean = 0
    for p in model.parameters():
        if p.grad is not None:
            grad_mean += p.grad.data.std().item()
    return grad_mean

In [131]:
#parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
#parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
#parser.add_argument('--resume', '-r', action='store_true', help='resume from checkpoint')
#args = parser.parse_args(argv[1:])
args = easydict.EasyDict({
    "lr": 0.1,
    "resume": '-r'
})

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
writer = SummaryWriter()


In [132]:
def visualize(model, epoch):
    writer.add_scalar('Gradient norm', compute_gradient_norm(model), epoch)
    writer.add_scalar('Gradient mean', compute_gradient_mean(model), epoch)
    writer.add_scalar('Gradient std', compute_gradient_mean(model), epoch)
    diction = get_grad_norm_by_param(model)
    for layer in diction:
        writer.add_scalar(layer, diction[layer], epoch)
    
#     for p in model.parameters():
#         if p.grad is not None:
#             flat=torch.flatten(p.grad.data)
#             print(flat.nelement())
#             for num in flat:
                #print(num.item())
                #writer.add_histogram('Gradient', num, epoch)
    

In [133]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [134]:
# Model
print('==> Building model..')
# net = VGG('VGG19')
net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
#net = EfficientNetB0()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# if args.resume:
#     # Load checkpoint.
#     print('==> Resuming from checkpoint..')
#     assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
#     checkpoint = torch.load('./checkpoint/ckpt.pth')
#     net.load_state_dict(checkpoint['net'])
#     best_acc = checkpoint['acc']
#     start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)

==> Building model..


In [135]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    visualize(net, epoch)
    hist_loss = np.array([])
    hist_accuracy = np.array([])
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        hist_loss = np.append(hist_loss, train_loss/(batch_idx+1))
        hist_accuracy = np.append(hist_accuracy, 100.*correct/total)
        
        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    writer.add_histogram('train loss', hist_loss, epoch)
    writer.add_histogram('train accuracy', hist_accuracy, epoch)

In [136]:

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
             # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

In [137]:
for epoch in range(start_epoch, start_epoch+3):
    train(epoch)
    test(epoch)


Epoch: 0
 [================================================================>]  Step: 112ms | Tot: 57s182ms | Loss: 1.973 | Acc: 29.136% (14568/5000 391/391 
 [================================================================>]  Step: 38ms | Tot: 4s79ms | Loss: 1.591 | Acc: 41.480% (4148/1000 100/100 ]  Step: 39ms | Tot: 174ms | Loss: 1.584 | Acc: 40.600% (203/50 5/100 .............................]  Step: 43ms | Tot: 346ms | Loss: 1.599 | Acc: 41.222% (371/90 9/10 10/100 11/100 =======>.........................................................]  Step: 38ms | Tot: 475ms | Loss: 1.593 | Acc: 41.750% (501/120 12/10 13/100 ==========>......................................................]  Step: 37ms | Tot: 672ms | Loss: 1.598 | Acc: 41.588% (707/170 17/100 .......................]  Step: 42ms | Tot: 845ms | Loss: 1.598 | Acc: 41.857% (879/210 21/100 30/100 ====================>............................................]  Step: 37ms | Tot: 1s339ms | Loss: 1.593 | Acc: 41.697% (1376/330 33

 [================================================================>]  Step: 38ms | Tot: 4s153ms | Loss: 1.188 | Acc: 57.300% (5730/1000 100/100 ..........................................................]  Step: 44ms | Tot: 191ms | Loss: 1.142 | Acc: 59.200% (296/50 5/100 .....................................................]  Step: 40ms | Tot: 232ms | Loss: 1.108 | Acc: 59.833% (359/60 6/100 ===>.............................................................]  Step: 42ms | Tot: 275ms | Loss: 1.119 | Acc: 59.714% (418/70 7/100 ................................]  Step: 42ms | Tot: 318ms | Loss: 1.146 | Acc: 58.375% (467/80 8/100 ...........]  Step: 46ms | Tot: 365ms | Loss: 1.160 | Acc: 58.333% (525/90 9/100 .............................]  Step: 40ms | Tot: 490ms | Loss: 1.161 | Acc: 57.917% (695/120 12/100 14/100 =========>.......................................................]  Step: 47ms | Tot: 626ms | Loss: 1.163 | Acc: 57.800% (867/150 15/100 =========>................................

1728
64
64
36864
64
64
36864
64
64
36864
64
64
36864
64
64
73728
128
128
147456
128
128
8192
128
128
147456
128
128
147456
128
128
294912
256
256
589824
256
256
32768
256
256
589824
256
256
589824
256
256
1179648
512
512
2359296
512
512
131072
512
512
2359296
512
512
2359296
512
512
5120
10
